In [1]:
!pip install accelerate transformers ftfy diffusers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 23.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import transformers
from diffusers import UNet2DConditionModel, VQDiffusionPipeline, DDIMScheduler, VQModel, AutoencoderKL

In [3]:
repo_id = 'CompVis/ldm-text2im-large-256'

In [4]:
# from diffusers import LDMTextToImagePipeline

# sd = LDMTextToImagePipeline.from_pretrained(repo_id)

In [5]:
# sd

In [8]:
from custom_pipeline import LDMSpeechToFacePipeline
from sf2f_encoder import SF2FEncoder

sf2f_encoder_kwargs = {
    'input_channel': 40,
    'channels': [256, 384, 576, 864],
    'output_channel': 512,
    'add_noise': False,
    'normalize': True,
    'return_seq': False,
    'inception_mode': True,
    }

voice_encoder = SF2FEncoder(sf2f_encoder_kwargs)
unet = UNet2DConditionModel.from_pretrained(repo_id, subfolder='unet', safetensors=True)
vqvae = AutoencoderKL.from_pretrained(repo_id, subfolder='vqvae')
scheduler = DDIMScheduler.from_pretrained(repo_id, subfolder='scheduler')

custom_pipeline = LDMSpeechToFacePipeline(unet=unet, vqvae=vqvae, scheduler=scheduler, sf2f=voice_encoder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


unet/config.json:   0%|          | 0.00/713 [00:00<?, ?B/s]

An error occurred while trying to fetch CompVis/ldm-text2im-large-256: CompVis/ldm-text2im-large-256 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


diffusion_pytorch_model.bin:   0%|          | 0.00/3.49G [00:00<?, ?B/s]

vqvae/config.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

An error occurred while trying to fetch CompVis/ldm-text2im-large-256: CompVis/ldm-text2im-large-256 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

The config attributes {'timestep_values': None} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


In [9]:
custom_pipeline

LDMSpeechToFacePipeline {
  "_class_name": "LDMSpeechToFacePipeline",
  "_diffusers_version": "0.29.2",
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "sf2f": [
    "sf2f_encoder",
    "SF2FEncoder"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vqvae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [10]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [11]:
print_trainable_parameters(voice_encoder)

trainable params: 4848192 || all params: 4848192 || trainable%: 100.00


In [12]:
unet

UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_fe

In [19]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['to_q', 'to_v'],
    lora_dropout=0.1,
    bias='none',
    modules_to_save=['unet_lora'],
)
lora_model = get_peft_model(unet, config)
print_trainable_parameters(lora_model)

trainable params: 862720 || all params: 874964804 || trainable%: 0.10
